In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Visual_tools as VTools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list)) # insert your file
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_2240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_2040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1640.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1440.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0640.raw', 'C://Users//jony//Programming//Python//

In [2]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(VTools)
import matplotlib.pyplot as plt
interval = 100 # Set your monitoring chunck size
file = file_path[-1] # Select the file
fReader = rawFileReader.fileReader(file) # load in the classs object
fReader.skip_events(900*interval)
print(file)
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents)
file_name = file.split("//")[-1][:-4] #must be a right file
    

import cv2
import os

image_folder = 'images_video'
video_name = f'{file_name}_bad.mp4'



with tqdm(total=max_process_event_chunk*interval, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        try:
            event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        except:
            print(f'Error at event chunk {processedEvents}')
            continue
        #Zone of Reconstruction
        if event_chunk:
            for evt_num, evt in enumerate(event_chunk):
                #[print(hit) for hit in VTools.all_hits_event(evt)]
                maps = VTools.hitHeatMap(evt)
                for rpc in range(6):
                    plt.imshow(maps[rpc], interpolation='nearest')
                    plt.title(f'Event {evt_num} RPC {rpc}')
                    plt.savefig(f'images_video/{processedEvents*100+10*evt_num+rpc}.png')
                pbar.update(1)

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape


video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

Skipping Events: 100%|██████████| 90000/90000 [00:10<00:00, 8412.45Events/s] 


C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw


Processing Events: 100%|██████████| 100/100 [36:10<00:00, 21.71s/Events]


In [32]:
import Atlas_tools as AtTools
importlib.reload(AtTools)
import pickle

anubis_file = 'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data(anubis_file, trigger_channel=4,bcr_channel=2)
analyser.printBCR(10)

New
Initial Time: 2024-06-27 16:20:18.092231
BCR(737854*25/32 ns, 2024-06-27 16:20:18.088189, 2024-06-27 16:20:18.088100,1) 
    Triggers: [])
BCR(917438*25/32 ns, 2024-06-27 16:20:18.088278, 2024-06-27 16:20:18.088100,2) 
    Triggers: [])
BCR(965950*25/32 ns, 2024-06-27 16:20:18.088367, 2024-06-27 16:20:18.088100,3) 
    Triggers: [])
Trigger 982398
Bcr 965950
BCID 514.0
Trigger(982398*25/32 ns, 1719501618.08838, 514.0)
Trigger 995372
Bcr 965950
BCID 919.4375
Trigger(995372*25/32 ns, 1719501618.08839, 919.4375)
BCR(31406*25/32 ns, 2024-06-27 16:20:18.088456, 2024-06-27 16:20:18.088100,4) 
    Triggers: [])
Trigger 131423
Bcr 31406
BCID 3125.53125
Trigger(131423*25/32 ns, 1719501618.0885346, 3125.53125)
BCR(145535*25/32 ns, 2024-06-27 16:20:18.088546, 2024-06-27 16:20:18.088100,5) 
    Triggers: [])
BCR(259518*25/32 ns, 2024-06-27 16:20:18.088634, 2024-06-27 16:20:18.088100,6) 
    Triggers: [])
Trigger 270335
Bcr 259518
BCID 338.03125
Trigger(270335*25/32 ns, 1719501618.0886428, 338.

In [25]:
import pandas
import Atlas_tools as AtTools
import pickle
importlib.reload(AtTools)
import numpy as np

an = AtTools.AtlasAnalyser()
with open('analyser.pkl', 'rb') as inp:
    an.anubis_data = pickle.load(inp)

#print(data["TimeStamp"][0:10])

atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//datafiles.root"
data = an.getAtlasData(atlas_file)
#an.atlas_data = an.atlas_data.sort_values(by=['TimeStamp'])
#print(an.atlas_data)
an.pairBcIdWithAtlasHit()



Atlas increase: (np.float64(1719505388.2931764), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.3053854), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.3120418), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.3207939), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.3535075), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.375872), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.4032607), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.405399), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.4074142), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.4117286), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.426826), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.4305167), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.436798), 1719505503.7113602)
Atlas increase: (np.float64(1719505388.4533868), 1719505503.7113602)
Atlas increase: (np.float64(1719505388

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
